<a href="https://colab.research.google.com/github/gojiteji/study_ReinforcementLearning_with_JAX/blob/main/Chapter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import jax
import jax.numpy as jnp

In [13]:
from jax import random
key = random.PRNGKey(0)  # Random seed is explicit in JAX
randoms = random.uniform(key, shape=(11,))

In [15]:
rewards = []
Q=0
for i in range(1,11):
  Q = Q + (randoms[i] - Q)/i
  print(Q)

0.13573384
0.40354812
0.51077765
0.46997517
0.44336867
0.48235852
0.41620627
0.41607106
0.3877441
0.37382877


In [16]:
rewards = []
Q=0
for i in range(1,11):
  Q += (randoms[i] - Q)/i
  print(Q)

0.13573384
0.40354812
0.51077765
0.46997517
0.44336867
0.48235852
0.41620627
0.41607106
0.3877441
0.37382877


In [39]:
import time
class Bandit:
  def __init__(self,arms=10):
    key = random.PRNGKey(0)
    self.rates = random.uniform(key, shape=(arms,1))# defalt win rate
  def play(self, arm):
    rate = self.rates[arm]
    key = random.PRNGKey(int(time.time()*1000))
    if rate > random.uniform(key, shape=(1,1)):# is it larger than win rate?
      return 1
    else:
      return 0

In [40]:
bandit = Bandit()
for i in range(10):
  print(bandit.play(0))

0
1
0
1
0
1
1
0
0
0
